### 머신러닝 프로세스 기반 
#### 수집된 (전처리된 데이터) -> 모델 -> 모델 학습 -> 예측(평가)
#### 데이터 : UCI 레포지토리에서 제공된, 유방암 진단 샘플 데이터를 이용
#### 로지스틱 회귀, 시그모이드 함수,
#### 성능 평가 지표 -> 오차행렬, 정밀도, 재현율, F1 스코어, ROC 기반, AUC 스코어
#### 예) 암이 아닌 환자, 암환자 예측
#### 행(실제)
#### 0  TN(True Negative)   FP (False Positive)
#### 1  FN(False Negative)  TP (True Positive)

### 용어정리
#### 로지스틱 회귀 : 분류에 사용하는 기법, 선형함수(직선)을 사용해서, 추세를 예측하는 선형회귀 
#### 선형회귀랑은 다르게, 참, 거짓을 분류 하는 목적 . 비선형 함수 (시그모이드 함수)
#### f(x) = 1 / ( 1 + e^-x )

#### 시그모이드 함수 기본형 -> s자형 
#### x 클수록 1에 근사
#### x 작을 수록 0에 근사


#### 오차 행렬

#### 행(실제 클래스)        (예측 부정)       (예측 긍정)       열(예측 클래스)
#### 0(실제부정)            TN                FP
#### 1(실제긍정)            FN                TP

#### 정확도 = 바르게 예측한 수 / 전체 데이터 수
#### 정확도 = (TN + TP) / (TN + TP + FN + TP)

#### Positive 예측 성능을 더 정밀하게 평가하는 지표
#### 사이킷 런 함수로 제공 : precision_score
#### 정밀도 = 참인 것의 수 (TP) / 예측이 긍정인 것 (FP + TP)

#### 암환자 진단 할 땐 정밀도가 더 중요 Positive 나온것 중에 실제 암이냐 아니냐?

#### 재현율 = 참인 것의 수 (TP) / 실제값이 긍정인 것 중 (FN + TP)
#### 실제 Positive인 데이터를 정확히 예측하는 평가 지표. 민감도라고도 하며, TPR (True Positive Rate)
#### 사이킷 런 함수로 제곡 : recall_score

#### F1 스코어 : 정밀도와 재현율을 결합한 평가지표, 상충관계 문제점을 고려해서, 저확한 평가에 사용됨 
#### F1_score = 2 * ( 정밀도 * 재현율) / (정밀도 + 재현율)

#### ROC 기반 AUC 스코어
#### ROC = 오차행렬에서, FPR(False Positive Rate)
#### FPR = FP / (TN + FP)  => 거짓인것 수 / 실제값이 거짓인것 중
#### -> 1에 가까울 수록 좋은 성능
#### -> 사이킷 런에서 함수 제공 : roc_auc_score


#### 유방암 데이터 샘플을 가지고, 실제 암환자, 모델을 학습 -> 학습된 모델을 평가
#### 평가의 예측값이 -> 셀제값과 어느정도 일치 하느지 여부, 우리가 확인

#### 보스턴 집값도 , 사이킷 런에서 내장된 데이터를 사용햇음 

In [3]:
!pip3 install scikit-learn==1.0.2

In [4]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer

In [5]:
b_cancer = load_breast_cancer()
print(b_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [6]:
b_cancer_df = pd.DataFrame(b_cancer.data, columns=b_cancer.feature_names)
# 새 컬럼 추가, 종속 변수인, 암의 진단 여부 컬럼 추가. 
b_cancer_df["diagnosis"] = b_cancer.target  # 1:악성(암) 2.양성(혹) 
b_cancer_df.head()

mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0                 0.07871  ...          17.33           184.60      2019.0   
1                 0.05667  ...          23.41           158.80      1956.0   
2                 0.05999  ...          25.53           152.50      1709.0   
3                 0.09744  ...          26.50            98.87       567.7   
4                 0.05883  ...          16.67           152.20      1575.0   

   worst smoothness  worst compactness  worst concavity  worst concave points  \
0            0.1622             0.6656           0.7119                0.2654   
1            0.1238             0.1866           0.2416                0.1860   
2            0.1444             0.4245           0.4504                0.2430   
3            0.2098             0.8663           0.6869                0.2575   
4            0.1374             0.2050           0.4000                0.1625   

   worst symmetry  worst fractal dimension  diagnosis  
0          0.4601                  0.11890          0  
1          0.2750                  0.08902          0  
2          0.3613                  0.08758          0  
3          0.6638                  0.17300          0  
4          0.2364                  0.07678          0  

[5 rows x 31 columns]

In [7]:
# 데이터 갯수 파악
print(f"샘플 데이터 크기 조회 : {b_cancer_df.shape}")

샘플 데이터 크기 조회 : (569, 31)


In [8]:
# 기본 속성 파악 (전처리 되어있는지 확인)
b_cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [9]:
# 로지스틱 회귀분석에 피쳐로 사용할 데이터를 평균 0 분산 1
# 정규 분포 형태로 변환 -> 사이킷 런에서 제공해주는 함수를 이용함
# 변환을 하는 이유, 데이터의  동일성, 분석을 효율적으로 하기 위해서 

# 정규화를 하는 이유
# 1) 서로다른 스케일을 가진 변수들을 , 동일한 청도로 맞춰서 작업 -> 정확도 향상
# 2) 알고리즘 성능 향샹
# 3) 이상치의 영향 감수
# 4) 튜닝시 사용이되는 방법중 하나가, 경사하강방법이 있는데 ,수렵 속도를 빠르게 해줘서 
# 전체적 성능이 좋아짐 

# 사이킷런에서 제공해서주는 정규화 도구
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# scaler 도구를 이용해서, 유방암 샘플 데이터를 정규화 작업 
b_cancer_scaled = scaler.fit_transform(b_cancer.data)
print(f"샘플데이터 정규화 전  b_cancer.data[0] 결과 확인 : {b_cancer.data[0]}")
print(f"샘플데이터 정규화 후  b_cancer_scaled[0] 결과 확인 : {b_cancer_scaled[0]}")

샘플데이터 정규화 전  b_cancer.data[0] 결과 확인 : [1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
샘플데이터 정규화 후  b_cancer_scaled[0] 결과 확인 : [ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


In [10]:
# 사이킷 런에서 제공하는 모델 불러와서 -> 학습 -> 학습된 모델이용해서 예측값 출력 -> 실제값과 비교분석

# 로지스틱 회귀 분석 도구
from sklearn.linear_model import LogisticRegression
# 훈련, 평가 데이터 분리
from sklearn.model_selection import train_test_split

# 독립 변수, 종속 변수("diagnosis") 분리
Y = b_cancer_df["diagnosis"]
# 정규화 된 샘플
X = b_cancer_scaled

In [11]:
# 훈련용, 테스트용으로 데이터를 분리하는 작업 7:3
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state = 0 )

In [12]:
print(f"X_train : {X_train.shape}, X_test : {X_test.shape}, Y_train : {Y_train.shape}, Y_test : {Y_test.shape}")

X_train : (398, 30), X_test : (171, 30), Y_train : (398,), Y_test : (171,)


In [13]:
# 모델 생성 
lr_b_cancer = LogisticRegression()

In [14]:
# 해당 모델 학습 시키기
lr_b_cancer.fit(X_train,Y_train)

LogisticRegression()

In [17]:
# 학습된 모델을 이용해서, 예측값 출력해보기
y_predict = lr_b_cancer.predict(X_test)

In [19]:
# 평가 분석 지표
# MSE : 오류 제곱 평균
# R^2 Score : 예측값 / 실제값 나눈 비율
# Y_test : 답지, y_predict : 상용이가 학습해서, 시험친 결과
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(Y_test, y_predict)
# 4 -> 루트 4 -> 2 값을 통일 시켰다. 나중에 스케일링을 많이 함 , 정규화 등으로 고쳐야
# 통계시 , 효율적으로 작업이 진행되므로, 그래서 ,이런 작업을 많이 함 
rmse = np.sqrt(mse)
r2_value = r2_score(Y_test,y_predict)

print(f"mse : {mse}, rmse : {rmse}, r2_value : {r2_value}")

mse : 0.023391812865497075, rmse : 0.1529438225803745, r2_value : 0.8994708994708995


In [20]:
#  오차 행렬을 이용해서 , 분석의 비교 지표값을 정밀도, 재현율, F1 스코어 , ROC와 AUC 스코어등을 이용해서
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [21]:
# 오차행렬 만들기 행(실제 값:Y_test) , 열 (예측 값:y_predict)
# 열 (예측 값:y_predict) , Negative , Positive 
# 행(실제 값:Y_test) Ne    TN         FP
# 행(실제 값:Y_test) Po    FN         TP
confusion_matrix(Y_test, y_predict)

array([[ 60,   3],
       [  1, 107]])

#### 정확도 = 바르게 예측한 수 / 전체 데이터 수
#### 정확도 = (TN + TP) / (TN + TP + FN + TP)

#### Positive 예측 성능을 더 정밀하게 평가하는 지표
#### 사이킷 런 함수로 제공 : precision_score
#### 정밀도 = 참인 것의 수 (TP) / 예측이 긍정인 것 (FP + TP)

#### 암환자 진단 할 땐 정밀도가 더 중요 Positive 나온것 중에 실제 암이냐 아니냐?

#### 재현율 = 참인 것의 수 (TP) / 실제값이 긍정인 것 중 (FN + TP)
#### 실제 Positive인 데이터를 정확히 예측하는 평가 지표. 민감도라고도 하며, TPR (True Positive Rate)
#### 사이킷 런 함수로 제곡 : recall_score

#### F1 스코어 : 정밀도와 재현율을 결합한 평가지표, 상충관계 문제점을 고려해서, 저확한 평가에 사용됨 
#### F1_score = 2 * ( 정밀도 * 재현율) / (정밀도 + 재현율)

#### ROC 기반 AUC 스코어
#### ROC = 오차행렬에서, FPR(False Positive Rate)
#### FPR = FP / (TN + FP)  => 거짓인것 수 / 실제값이 거짓인것 중
#### -> 1에 가까울 수록 좋은 성능
#### -> 사이킷 런에서 함수 제공 : roc_auc_score

In [26]:
# 정확도 (TN + TP) / (TN + TP + FN + TP)
accuracy = accuracy_score(Y_test, y_predict)
print(f"accuarcy(정확도) 값 : {accuracy}")

accuarcy(정확도) 값 : 0.9766081871345029


In [24]:
# 정밀도 (TP) / (FP + TP)
precision = precision_score(Y_test, y_predict)
print(f"precision(정밀도) 값 : {precision}")

precision(정밀도) 값 : 0.9727272727272728


In [25]:
# 재현율 (TP) / (FN + TP)
recall = recall_score(Y_test, y_predict)
print(f"recall(재현율) 값 : {recall}")

recall(재현율) 값 : 0.9907407407407407


In [28]:
# F1 스코어
f1 = f1_score(Y_test, y_predict)
print(f"f1_score 값 : {f1}")

f1_score 값 : 0.981651376146789


In [30]:
# ROC 기반 AUC 스코어
roc_auc = roc_auc_score(Y_test, y_predict)
print(f"roc_auc_score 값 : {roc_auc}")

roc_auc_score 값 : 0.9715608465608465


In [33]:
# 소숫점 3자리 까지 
print("정확도 : {0:.3f}/ 정밀도 : {1:.3f}/ 재현율 : {2:.3f}/ F1 스코어 : {3:.3f}/ ROC 기반 AUC 스코어 : {4:.3f}".format(accuracy,precision,recall,f1,roc_auc))

정확도 : 0.977/ 정밀도 : 0.973/ 재현율 : 0.991/ F1 스코어 : 0.982/ ROC 기반 AUC 스코어 : 0.972
